In [ ]:
!pip install google-generativeai

In [ ]:
import os
import google.generativeai as genai
import pandas as pd
import time
from google.api_core.exceptions import TooManyRequests

In [ ]:
api_key = ""
genai.configure(api_key=api_key)
model = genai.GenerativeModel('models/gemini-2.5-flash-preview-04-17')

In [ ]:
REQUESTS_PER_MINUTE = 10
DELAY_BETWEEN_REQUESTS = 60 / REQUESTS_PER_MINUTE

def make_request(prompt, model, model_name=model):
    response = model.generate_content(prompt)
    return response.text

##Direct and COT prompting

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Stanford NLP/prompts-step2 - prompts.csv")
df

,Task,direct prompt,cot
0,Write a Python function using TensorFlow to cr...,Write a Python function using TensorFlow to cr...,Write a Python function using TensorFlow to cr...
1,Write a Python script using PyTorch to train a...,Write a Python script using PyTorch to train a...,Write a Python script using PyTorch to train a...
2,Create a scikit-learn pipeline that generates ...,Create a scikit-learn pipeline that generates ...,Create a scikit-learn pipeline that generates ...
3,Write a Python function that uses Hugging Face...,Write a Python function that uses Hugging Face...,Write a Python function that uses Hugging Face...
4,Build a TensorFlow model for image classificat...,Build a TensorFlow model for image classificat...,Build a TensorFlow model for image classificat...
5,Write a PyTorch class that subclasses torch.ut...,Write a PyTorch class that subclasses torch.ut...,Write a PyTorch class that subclasses torch.ut...
6,Write a Python script that loads the Iris data...,Write a Python script that loads the Iris data...,Write a Python script that loads the Iris data...
7,Write a Java program using the Hadoop MapReduc...,Write a Java program using the Hadoop MapReduc...,Write a Java program using the Hadoop MapReduc...


In [ ]:

if "direct_response" not in df.columns:
    df["direct_response"] = ""
if "cot_response" not in df.columns:
    df["cot_response"] = ""

for index, row in df.iterrows():
    print(f"Processing row {index+1}/{len(df)}...")

    try:
        prompt_direct = row['direct prompt']
        response_direct = make_request(prompt_direct, model)
        print(response_direct)
        df.at[index, "direct_response"] = response_direct
    except TooManyRequests:
        print("Rate limit hit during direct prompt. Skipping.")
        df.at[index, "direct_response"] = "Error: Rate limit exceeded."
    except Exception as e:
        print(f"Error generating direct response: {e}")
        df.at[index, "direct_response"] = f"Error: {e}"

    time.sleep(DELAY_BETWEEN_REQUESTS)

    try:
        prompt_cot = row['cot']
        response_cot = make_request(prompt_cot, model)
        print(response_cot)
        df.at[index, "cot_response"] = response_cot
    except TooManyRequests:
        print("Rate limit hit during CoT prompt. Skipping.")
        df.at[index, "cot_response"] = "Error: Rate limit exceeded."
    except Exception as e:
        print(f"Error generating CoT response: {e}")
        df.at[index, "cot_response"] = f"Error: {e}"

    time.sleep(DELAY_BETWEEN_REQUESTS)

    df.to_csv("/content/drive/MyDrive/Stanford NLP/prompts-step2 - responses.csv", index=False)


Processing row 1/8...
```python
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def train_california_housing_linear_model():
    # Load the dataset
    housing = fetch_california_housing()
    X, y = housing.data, housing.target

    # Preprocess the data (scaling)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y_scaled = y # For simplicity, we won't scale the target here, model will learn direct mapping

    # Split data (optional but good practice)
    # X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
    # Using all data for training as requested implicitly

    # Define the model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=1, input_shape=(X_scaled.shape[1],))
    ])

    # Compile the model
    model.compile(optimizer='adam', 

##Few shot

In [ ]:
df_fs = pd.read_csv("/content/drive/MyDrive/Stanford NLP/few_shot_prompts.csv")
df_fs

,prompt
0,Create a python script that uses tensorflow to...
1,Write a Python script using PyTorch to train a...
2,Create a scikit learn pipeline that builds sta...
3,In this example we learn to identify past vs p...
4,Build an image recognition tensorflow model to...
5,Write a PyTorch class that creates custom data...
6,Write a Python script that loads the UCI handw...
7,Write a Java program using the Hadoop MapReduc...


In [ ]:
if "few_shot" not in df_fs.columns:
    df_fs["few_shot"] = ""


for index, row in df_fs.iterrows():
    print(f"Processing row {index+1}/{len(df_fs)}...")

    try:
        prompt_fs = row['prompt']
        response_fs = make_request(prompt_fs,model)
        df_fs.at[index, "few_shot"] = response_fs
    except Exception as e:
        print(f"Error generating direct response: {e}")
        df_fs.at[index, "few_shot"] = f"Error: {e}"

    time.sleep(DELAY_BETWEEN_REQUESTS)

    df_fs.to_csv("/content/drive/MyDrive/Stanford NLP/prompts-step2 - responses_few_Shot_gemini.csv", index=False)

Processing row 1/8...
Processing row 2/8...
Processing row 3/8...
Processing row 4/8...
Processing row 5/8...
Processing row 6/8...
Processing row 7/8...
Processing row 8/8...


##API extraction

In [ ]:
df_api = pd.read_csv("/content/drive/MyDrive/Stanford NLP/api_documentation.csv")
df_api

,API Component,Documentation
0,tensorflow,"Top-level module of TensorFlow. By convention,..."
1,tensorflow.keras,DO NOT EDIT. This file was autogenerated. Do ...
2,Sequential,`Sequential` groups a linear stack of layers i...
3,Dense,Just your regular densely-connected NN layer. ...
4,Input,Used to instantiate a Keras tensor. A Keras t...
5,Concatenate,Concatenates a list of inputs. It takes as in...
6,fetch_california_housing,Load the California housing dataset (regressio...
7,make_classification,Generate a random n-class classification probl...
8,train_test_split,Split arrays or matrices into random train and...
9,StandardScaler,Standardize features by removing the mean and ...


In [ ]:
prompt="Given the following API documentation, extract a structured representation of the API, including types, functions, and their relationships: [API documentation] Provide the structured representation in JSON format."

In [ ]:
json_outputs = []

for index, row in df_api.iterrows():
    print(f"Processing row {index+1}/{len(df_api)}...")

    try:
        prompt = (
            f"Given the following API documentation, extract a structured representation of the API, including types, functions, and their relationships:\n"
            f"{row['Documentation'].strip()}\n\n"
            f"Provide the structured representation in JSON format."
        )

        response = make_request(prompt, model)
    except Exception as e:
        print(f"Error generating response: {e}")
        response = f"Error: {e}"

    json_outputs.append(response)

    time.sleep(DELAY_BETWEEN_REQUESTS)

Processing row 1/33...
Processing row 2/33...
Processing row 3/33...
Processing row 4/33...
Processing row 5/33...
Processing row 6/33...
Processing row 7/33...
Processing row 8/33...
Processing row 9/33...
Processing row 10/33...
Processing row 11/33...
Processing row 12/33...
Processing row 13/33...
Processing row 14/33...
Processing row 15/33...
Processing row 16/33...
Processing row 17/33...
Processing row 18/33...
Processing row 19/33...
Processing row 20/33...
Processing row 21/33...
Processing row 22/33...
Processing row 23/33...
Processing row 24/33...
Processing row 25/33...
Processing row 26/33...
Processing row 27/33...
Processing row 28/33...
Processing row 29/33...
Processing row 30/33...
Processing row 31/33...
Processing row 32/33...
Processing row 33/33...


In [ ]:
df_results = pd.DataFrame({'api_extract': json_outputs})

output_path = "/content/drive/MyDrive/Stanford NLP/api_extraction_results.csv"
df_results.to_csv(output_path, index=False)

##Neurosymbolic Generation

###Neural suggestion

In [ ]:
df_api = pd.read_csv("/content/drive/MyDrive/Stanford NLP/api_documentation_new.csv")
df_api

,API Component,Documentation,api_extract_gemini,api_extract_gpt
0,tensorflow,"Top-level module of TensorFlow. By convention,...","```json\n{\n ""module"": {\n ""name"": ""tf"",\n...","```json\n{\n ""module"": ""tensorflow"",\n ""alia..."
1,tensorflow.keras,DO NOT EDIT. This file was autogenerated. Do ...,"```json\n{\n ""types"": {\n ""SomeType"": {\n ...",Of course! Please provide the API documentatio...
2,Sequential,`Sequential` groups a linear stack of layers i...,"```json\n{\n ""description"": ""API representati...",Certainly! Here's a structured representation ...
3,Dense,Just your regular densely-connected NN layer. ...,"```json\n{\n ""name"": ""Dense"",\n ""type"": ""Lay...",Here is a structured representation of the API...
4,Input,Used to instantiate a Keras tensor. A Keras t...,"```json\n{\n ""types"": [\n {\n ""name"":...",To extract the structured representation of th...
5,Concatenate,Concatenates a list of inputs. It takes as in...,"```json\n{\n ""name"": ""Concatenate"",\n ""type""...","```json\n{\n ""API"": ""Concatenate"",\n ""Type"":..."
6,fetch_california_housing,Load the California housing dataset (regressio...,"```json\n{\n ""module"": ""sklearn.datasets"",\n ...",Here's a structured representation of the API ...
7,make_classification,Generate a random n-class classification probl...,"```json\n{\n ""function"": {\n ""name"": ""make...",Here's a structured representation of the API ...
8,train_test_split,Split arrays or matrices into random train and...,"```json\n{\n ""function"": {\n ""name"": ""trai...",Below is the structured representation of the ...
9,StandardScaler,Standardize features by removing the mean and ...,"```json\n{\n ""classes"": [\n {\n ""name...","```json\n{\n ""class"": ""StandardScaler"",\n ""d..."


In [ ]:
api_dict = dict(zip(df_api['API Component'], df_api['api_extract_gemini']))
api_dict.keys()

dict_keys(['tensorflow', 'tensorflow.keras', 'Sequential', 'Dense', 'Input', 'Concatenate', 'fetch_california_housing', 'make_classification', 'train_test_split', 'StandardScaler', 'LogisticRegression', 'make_pipeline', 'PCA', 'torch', 'torch.nn', 'torch.optim', 'torchvision', 'torchvision.transforms', 'torch.nn.functional', 'Dataset', 'DataLoader', 'torchvision.datasets', 'ToTensor', 'numpy', 'pandas', 'matplotlib.pyplot', 'AutoTokenizer', 'DataCollatorWithPadding', 'AutoModelForSequenceClassification', 'TrainingArguments', 'Trainer', 'notebook_login', 'load_dataset'])

In [ ]:
api_dict

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Stanford NLP/dataset.csv")
df

,Task,Code,API doc,Few shot t1,Few shot c1,Few shot t2,Few shot c2,Unnamed: 7
0,Write a Python function using TensorFlow to cr...,import tensorflow as tf\nfrom tensorflow impor...,import tensorflow as tf\nfrom tensorflow impor...,Create a python script that uses tensorflow to...,\nimport numpy as np\nimport tensorflow as tf\...,Write a Python function using TensorFlow to cr...,pip install -q seaborn\n\nimport matplotlib.py...,NaN
1,Write a Python script using PyTorch to train a...,import torch\nimport torch.nn as nn\nimport to...,import torch.nn as nn\nimport torch.optim as o...,Write a Python script using PyTorch to train a...,import numpy as np\nimport pandas as pd\nimpor...,Write a Python script using PyTorch to train a...,from torch.nn import Module\nfrom torch.nn imp...,\n
2,Create a scikit-learn pipeline that generates ...,from sklearn.datasets import make_classificati...,from sklearn.datasets import make_classificati...,Create a scikit learn pipeline that builds sta...,from sklearn.pipeline import Pipeline\nfrom sk...,Create a scikit learn pipeline that builds sta...,from sklearn.pipeline import Pipeline\nfrom sk...,NaN
3,Write a Python function that uses Hugging Face...,from transformers import AutoTokenizer\nfrom t...,from transformers import AutoTokenizer\nfrom t...,In this example we learn to identify past vs p...,"\nfrom typing import Callable, Dict, Sequence,...",Write a python function that uses hugging face...,\nimport gradio as gr\nfrom transformers impor...,NaN
4,Build a TensorFlow model for image classificat...,import tensorflow as tf\nimport numpy as np\n\...,import tensorflow as tf\nimport numpy as np\nt...,Build an image recognition tensorflow model to...,import matplotlib.pyplot as plt \nimport numpy...,Create an image classification model using ten...,import tensorflow as tf \n\n# Display the ve...,NaN
5,Write a PyTorch class that subclasses torch.ut...,from torch.utils.data import Dataset\nfrom tor...,from torch.utils.data import Dataset\nfrom tor...,Write a PyTorch class that creates custom data...,\nimport os\nimport pandas as pd\nfrom torchvi...,Write a PyTorch class that subclasses torch.ut...,import torch\nfrom torch.utils.data import Dat...,NaN
6,Write a Python script that loads the Iris data...,\nfrom sklearn import datasets\nfrom sklearn.d...,\nfrom sklearn import datasets\nfrom sklearn.d...,Write a Python script that loads the UCI handw...,import matplotlib.pyplot as plt\nimport numpy ...,Write a Python script that uses the dataset gi...,import numpy as np\nimport pandas as pd\nfrom ...,NaN
7,Write a Java program using the Hadoop MapReduc...,Mapper Code\nimport java.io.IOException; ...,NaN,Write a Java program using the Hadoop MapReduc...,import java.io.IOException;\nimport java.util....,Write a Java program using the Hadoop MapReduc...,package hadoop; \n\nimport java.util.*; \n\nim...,\n


In [ ]:
output=[]

for index, row in df.iterrows():
    print(f"Processing row {index+1}/{len(df)}...")

    try:
        prompt = (
            f"Suggest an API usage pattern for the following task: :\n"
            f"{row['Task'].strip()}\n\n"
            f"Based on the API structure provided below:\n"
            f"{api_dict}\n\n"
            f"You can choose anything from given data or select a new API, if the provided one's are not able to fulfill the task."
        )

        response = make_request(prompt,model)
    except Exception as e:
        print(f"Error generating response: {e}")
        response = f"Error: {e}"

    output.append(response)

    time.sleep(DELAY_BETWEEN_REQUESTS)

In [ ]:
df_results = pd.DataFrame({'neural_gemini': output})

output_path = "/content/drive/MyDrive/Stanford NLP/neural_results.csv"
df_results.to_csv(output_path, index=False)

Generalization check

In [ ]:
generlaization_data = ["Create a python script that uses imbalanced-learn to do a binary classification model on Breast Cancer Wisconsin (Diagnostic) Dataset (WDBC), where your major class to minor class ratio is 99:1. You should use techniques like SMOTE to balance the dataset. Use a general model like XGboost. Do an analysis of metrics (like accuracy, precision, recall, f1 score, auc score and confusion matrix) before and after using imbalanced-learn techniques. ","Write a script in python that uses tensorflow and keras library. Your task is to work on Semantic Segmentation using Oxford Pets Dataset and a U-Net Model. The semantic segmentation task would be to differentiate between background and the pet. Also calculate pixel-wise classification accuracy, model loss and IoU."]

In [ ]:
generlaization_output=[]
for i in range(len(generlaization_data)):
    print(f"Processing row {i+1}/{len(generlaization_data)}...")
    try:
        prompt = (
            f"Suggest an API usage pattern for the following task: :\n"
            f"{generlaization_data[i].sttrip()}\n\n"
            f"Based on the API structure provided below:\n"
            f"{api_dict}\n\n"
            f"You can choose anything from given data or select a new API, if the provided one's are not able to fulfill the task."
        )

        response = make_request(prompt,model)
    except Exception as e:
        print(f"Error generating response: {e}")
        response = f"Error: {e}"

    generlaization_output.append(response)

###symbolic checking

In [ ]:
df_api = pd.read_csv("/content/drive/MyDrive/Stanford NLP/api_documentation_new.csv")
df_api

,API Component,Documentation,api_extract_gemini,api_extract_gpt
0,tensorflow,"Top-level module of TensorFlow. By convention,...","```json\n{\n ""module"": {\n ""name"": ""tf"",\n...","```json\n{\n ""module"": ""tensorflow"",\n ""alia..."
1,tensorflow.keras,DO NOT EDIT. This file was autogenerated. Do ...,"```json\n{\n ""types"": {\n ""SomeType"": {\n ...",Of course! Please provide the API documentatio...
2,Sequential,`Sequential` groups a linear stack of layers i...,"```json\n{\n ""description"": ""API representati...",Certainly! Here's a structured representation ...
3,Dense,Just your regular densely-connected NN layer. ...,"```json\n{\n ""name"": ""Dense"",\n ""type"": ""Lay...",Here is a structured representation of the API...
4,Input,Used to instantiate a Keras tensor. A Keras t...,"```json\n{\n ""types"": [\n {\n ""name"":...",To extract the structured representation of th...
5,Concatenate,Concatenates a list of inputs. It takes as in...,"```json\n{\n ""name"": ""Concatenate"",\n ""type""...","```json\n{\n ""API"": ""Concatenate"",\n ""Type"":..."
6,fetch_california_housing,Load the California housing dataset (regressio...,"```json\n{\n ""module"": ""sklearn.datasets"",\n ...",Here's a structured representation of the API ...
7,make_classification,Generate a random n-class classification probl...,"```json\n{\n ""function"": {\n ""name"": ""make...",Here's a structured representation of the API ...
8,train_test_split,Split arrays or matrices into random train and...,"```json\n{\n ""function"": {\n ""name"": ""trai...",Below is the structured representation of the ...
9,StandardScaler,Standardize features by removing the mean and ...,"```json\n{\n ""classes"": [\n {\n ""name...","```json\n{\n ""class"": ""StandardScaler"",\n ""d..."


In [ ]:
output = []

for index, row in df_api.iterrows():
    print(f"Processing row {index+1}/{len(df_api)}...")

    try:
        prompt = (
            f"Given the API Component:\n"
            f"{row['API Component'].strip()}\n"
            f"Use the given extracted API structure and generate rule based checking constraints for each of the components. Just write the important constraints to check in a bulleted format.\n"
            # f"Use the given extracted API structure and generate rule based checking constraints for each of the components by writing a python code to check those constraints.\n"
            f"{row['api_extract_gemini']}\n\n"
        )

        response = make_request(prompt, model)
    except Exception as e:
        print(f"Error generating response: {e}")
        response = f"Error: {e}"

    output.append(response)

    time.sleep(DELAY_BETWEEN_REQUESTS)

Processing row 1/33...
Processing row 2/33...
Processing row 3/33...
Processing row 4/33...
Processing row 5/33...
Processing row 6/33...
Processing row 7/33...
Processing row 8/33...
Processing row 9/33...
Processing row 10/33...
Processing row 11/33...
Processing row 12/33...
Processing row 13/33...
Processing row 14/33...
Processing row 15/33...
Processing row 16/33...
Processing row 17/33...
Processing row 18/33...
Processing row 19/33...
Processing row 20/33...
Processing row 21/33...
Processing row 22/33...
Processing row 23/33...
Processing row 24/33...
Processing row 25/33...
Processing row 26/33...
Processing row 27/33...
Processing row 28/33...
Processing row 29/33...
Processing row 30/33...
Processing row 31/33...
Processing row 32/33...
Processing row 33/33...


In [ ]:
df_results = pd.DataFrame({'gemini_symbolic': output})

output_path = "/content/drive/MyDrive/Stanford NLP/symbolic_gemini.csv"
df_results.to_csv(output_path, index=False)

###Final code generation

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Stanford NLP/dataset.csv")
df

,Task,Code,API doc,Few shot t1,Few shot c1,Few shot t2,Few shot c2,Unnamed: 7
0,Write a Python function using TensorFlow to cr...,import tensorflow as tf\nfrom tensorflow impor...,import tensorflow as tf\nfrom tensorflow impor...,Create a python script that uses tensorflow to...,\nimport numpy as np\nimport tensorflow as tf\...,Write a Python function using TensorFlow to cr...,pip install -q seaborn\n\nimport matplotlib.py...,NaN
1,Write a Python script using PyTorch to train a...,import torch\nimport torch.nn as nn\nimport to...,import torch.nn as nn\nimport torch.optim as o...,Write a Python script using PyTorch to train a...,import numpy as np\nimport pandas as pd\nimpor...,Write a Python script using PyTorch to train a...,from torch.nn import Module\nfrom torch.nn imp...,\n
2,Create a scikit-learn pipeline that generates ...,from sklearn.datasets import make_classificati...,from sklearn.datasets import make_classificati...,Create a scikit learn pipeline that builds sta...,from sklearn.pipeline import Pipeline\nfrom sk...,Create a scikit learn pipeline that builds sta...,from sklearn.pipeline import Pipeline\nfrom sk...,NaN
3,Write a Python function that uses Hugging Face...,from transformers import AutoTokenizer\nfrom t...,from transformers import AutoTokenizer\nfrom t...,In this example we learn to identify past vs p...,"\nfrom typing import Callable, Dict, Sequence,...",Write a python function that uses hugging face...,\nimport gradio as gr\nfrom transformers impor...,NaN
4,Build a TensorFlow model for image classificat...,import tensorflow as tf\nimport numpy as np\n\...,import tensorflow as tf\nimport numpy as np\nt...,Build an image recognition tensorflow model to...,import matplotlib.pyplot as plt \nimport numpy...,Create an image classification model using ten...,import tensorflow as tf \n\n# Display the ve...,NaN
5,Write a PyTorch class that subclasses torch.ut...,from torch.utils.data import Dataset\nfrom tor...,from torch.utils.data import Dataset\nfrom tor...,Write a PyTorch class that creates custom data...,\nimport os\nimport pandas as pd\nfrom torchvi...,Write a PyTorch class that subclasses torch.ut...,import torch\nfrom torch.utils.data import Dat...,NaN
6,Write a Python script that loads the Iris data...,\nfrom sklearn import datasets\nfrom sklearn.d...,\nfrom sklearn import datasets\nfrom sklearn.d...,Write a Python script that loads the UCI handw...,import matplotlib.pyplot as plt\nimport numpy ...,Write a Python script that uses the dataset gi...,import numpy as np\nimport pandas as pd\nfrom ...,NaN
7,Write a Java program using the Hadoop MapReduc...,Mapper Code\nimport java.io.IOException; ...,NaN,Write a Java program using the Hadoop MapReduc...,import java.io.IOException;\nimport java.util....,Write a Java program using the Hadoop MapReduc...,package hadoop; \n\nimport java.util.*; \n\nim...,\n


In [ ]:
tasks=df['Task'].to_list()
tasks

['Write a Python function using TensorFlow to create and compile a simple linear regression model with one dense layer. Train it on the classic California housing dataset.\n',
 'Write a Python script using PyTorch to train a convolutional neural network (CNN) on the Cifar 10 dataset. Include data loading, model definition, training loop, and evaluation',
 'Create a scikit-learn pipeline that generates a random n-class classification problem and creates a pipeline to compute the mean and standard deviation on a training set so as to be able to later re-apply the same transformation on the testing set. Train the model Fit the dataset using the same pipeline and compute score on the test set. ',
 'Write a Python function that uses Hugging Face to train a model(distilbert/distilbert-base-uncaseddistilbert/distilbert-base-uncased) using the IMDb Reviews dataset. Its  binary classification task. Involve any necessary preprocessing steps if required.',
 'Build a TensorFlow model for image cla

In [ ]:
df_neural = pd.read_csv("/content/drive/MyDrive/Stanford NLP/neural_results_gemini.csv")
df_neural

,neural_gemini
0,```python\nimport tensorflow as tf\nfrom sklea...
1,```python\nimport torch\nimport torch.nn as nn...
2,"Okay, here is an API usage pattern using sciki..."
3,```python\nimport torch\nfrom datasets import ...
4,"Okay, let's outline the API usage pattern for ..."
5,```python\nimport torch\nimport torch.utils.da...
6,```python\nimport matplotlib.pyplot as plt # ...
7,"Okay, the provided API structures (`tensorflow..."


In [ ]:
output = []

for index, row in df_neural.iterrows():
    print(f"Processing row {index+1}/{len(df)}...")

    try:
        prompt = (
            f"Generate complete code that solves the following task using the synthesized API usage pattern:\n"
            f"{tasks[index]}\n"
            f"{df_neural['neural_gemini']}\n\n"
        )

        response = make_request(prompt,model)
    except Exception as e:
        print(f"Error generating response: {e}")
        response = f"Error: {e}"

    output.append(response)

    time.sleep(DELAY_BETWEEN_REQUESTS)

Processing row 1/8...
Processing row 2/8...
Processing row 3/8...
Processing row 4/8...
Processing row 5/8...
Processing row 6/8...
Processing row 7/8...
Processing row 8/8...


In [ ]:
df_results = pd.DataFrame({'final_gemini': output})

output_path = "/content/drive/MyDrive/Stanford NLP/final_generated_code_gemini.csv"
df_results.to_csv(output_path, index=False)